In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [252]:
df = pd.read_csv('/Users/siboraseranaj/Desktop/project/STA440- Project/IST_corrected1.csv')

/Users/siboraseranaj/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [271]:
# result at six months
df["OCCODE"].unique()

array([2, 4, 3, 9, 1, 0])

In [272]:
# My question: for people given this medicine, who dies and who doesnt - with the measurment at 6 months 
# for this first question I need: the time of death(TD), did they die or not(OCCODE), and other variables that I will choose: 
# some physical characteristics, recurrent stroke or not

# My second question is: for people given this medicine, who will get new stroke and who will not - 
# for that I will look at any type of stroke,  DRSISC , and manually input the date of that happening. I guess other people would be censored


# known to be dead or alive at 14 days - this is my status set14, you have to ask professor for this
df["RDELAY"].unique()

# I also need the time of death during those 14 days

array([17, 10, 43,  6, 20, 39,  4, 24, 18, 13, 14, 16,  5, 12,  8, 29, 21,
       22,  2, 35, 45, 19,  3, 11,  9, 23, 26, 47, 25, 15, 28, 40, 30, 32,
       36, 34, 42, 38, 27,  7, 46, 48, 33, 37, 31, 41, 44,  1])

In [273]:
# stroke columns 'DRSISC', "DRSH", "DRSUNK", "DPE"

columns = ['RDELAY', 'SEX','AGE','RCONSC','RVISINF','RSBP','RDEF1',
           'RDEF2','RDEF3','RDEF4','RDEF5','RDEF6','RDEF7',
           'RDEF8','STYPE','RXASP','RATRIAL','RXHEP','TD','OCCODE','DRSISC', "DRSH", "DRSUNK", "DPE"]

In [274]:
obj1_df = df[columns]

In [275]:
def change_OCCODE(row):
    if row == 1:
        return 1
    elif row == 9 or row == 0:
        return None
    else:
        return 0

obj1_df['OCCODE'] = obj1_df['OCCODE'].apply(lambda x: change_OCCODE(x))

<ipython-input-275-7b95df005131>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obj1_df['OCCODE'] = obj1_df['OCCODE'].apply(lambda x: change_OCCODE(x))


In [276]:
obj1_df = obj1_df.dropna(subset=['OCCODE']).reset_index(drop = True)

In [277]:
def stroke_func(row):
    if row[0] == 'Y' or row[1] == 'Y' or row[2] == 'Y' or row[3] == 'Y':
        return 1
    else:
        return 0
    
obj1_df['recurrentstroke'] = obj1_df[['DRSISC', "DRSH", "DRSUNK", "DPE"]].apply(lambda x: stroke_func(x),axis = 1)

In [278]:
#def change_DF(row):
 #   if row[0] == 1 and (row[1]) > 185:
  #      return str(row[1])+'+'
  #  else:
   #     return str(row[1])
    
#obj1_df['TD'] = obj1_df[['OCCODE','TD']].apply(lambda x: change_DF(x),axis = 1)



In [306]:
obj_1_1 = obj1_df.copy()

In [280]:
def treatment(row):
    if row[0] == 'Y' and row[1] == 'N':
        return 'ASP'
    elif row[0] == 'N' and (row[1] == 'H' or row[1] == 'L' or row[1] == 'M'):
        return 'HEP'
    elif row[0] == 'Y' and (row[1] == 'H' or row[1] == 'L' or row[1] == 'M'):
        return 'BOTH'
    else:
        return 'NONE'

obj1_df['trt'] = obj1_df[['RXASP','RXHEP']].apply(lambda x: treatment(x), axis = 1)    

In [281]:
# only people with aspirin or heparin 9146 patients
HEPorASP = obj1_df.loc[((obj1_df["trt"] == "ASP") | (obj1_df["trt"] == "HEP"))]

In [282]:
HEPorASP = HEPorASP.dropna(subset=['RATRIAL']).reset_index(drop = True)

In [283]:
len(HEPorASP)

9146

In [284]:
# numerical value for amount of symptoms presented at admission
def count_def(row):
    count = 0
    for i in range(len(row)):
        if row[i] == 'Y':
            count += 1
    return count

HEPorASP['physicaldeficit'] = HEPorASP[['RDEF1', 'RDEF2',
       'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8']].apply(lambda x: count_def(x),axis = 1)

In [285]:
print(HEPorASP.columns)

Index(['RDELAY', 'SEX', 'AGE', 'RCONSC', 'RVISINF', 'RSBP', 'RDEF1', 'RDEF2',
       'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8', 'STYPE', 'RXASP',
       'RATRIAL', 'RXHEP', 'TD', 'OCCODE', 'DRSISC', 'DRSH', 'DRSUNK', 'DPE',
       'recurrentstroke', 'trt', 'physicaldeficit'],
      dtype='object')


In [286]:
HEPorASP = HEPorASP.rename(columns={'RVISINF':"infarct",
                       'RSBP':"bloodpressure" , "OCCODE": "death", "SEX":"sex", "AGE":"age",
                       'STYPE':"strokesubtype", 'TD':"timedeath",
                       'RDELAY':"timedelay",'RCONSC':'concious', 'RATRIAL' : 'atrialfibrillation'})


In [287]:
HEPorASP.drop(['RDEF1','RDEF2','RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8', 'RXASP', "RXHEP", 'DRSISC', 'DRSH', 'DRSUNK','DPE'], inplace=True, axis=1)

In [289]:
HEPorASP.head()

,timedelay,sex,age,concious,infarct,bloodpressure,strokesubtype,atrialfibrillation,timedeath,death,recurrentstroke,trt,physicaldeficit
0,20,F,64,F,N,150,TACS,Y,233.0,0.0,0,ASP,4
1,19,M,54,F,N,160,POCS,N,233.0,0.0,0,ASP,2
2,20,F,79,D,N,175,TACS,N,29.0,1.0,0,HEP,6
3,24,M,62,F,N,150,TACS,N,184.0,0.0,0,HEP,3
4,29,F,64,D,Y,130,PACS,N,186.0,0.0,0,ASP,3


In [290]:
HEPorASP.to_csv('/Users/siboraseranaj/Desktop/project/STA440- Project/HEPorASP.csv')

# Heparin high or low

In [307]:
obj_1_1["RXHEP"].unique()

array(['N', 'L', 'H', 'M'], dtype=object)

In [308]:
obj_1_1 = obj_1_1[obj_1_1["RXASP"] == "N"].reset_index(drop = True)

In [309]:
obj_1_1 = obj_1_1[obj_1_1["RXHEP"] != "N"].reset_index(drop = True)

In [310]:
def trthep(row):
    if row == "L":
        return "L"
    else:
        return "H"
    
obj_1_1["RXHEP"] = obj_1_1["RXHEP"].apply(lambda x : trthep(x))

In [311]:
obj_1_1 = obj_1_1.dropna(subset=['RATRIAL']).reset_index(drop = True)

In [312]:
obj_1_1['count_def'] = obj_1_1[['RDEF1', 'RDEF2',
       'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8']].apply(lambda x: count_def(x),axis = 1)
obj_1_1.head()

,RDELAY,SEX,AGE,RCONSC,RVISINF,RSBP,RDEF1,RDEF2,RDEF3,RDEF4,...,RXHEP,TD,OCCODE,DRSISC,DRSH,DRSUNK,DPE,recurrentstroke,trt,count_def
0,20,F,79,D,N,175,Y,Y,Y,Y,...,L,29.0,1.0,N,N,N,N,0,HEP,6
1,24,M,62,F,N,150,N,Y,Y,Y,...,H,184.0,0.0,N,N,N,N,0,HEP,3
2,2,F,90,F,N,200,N,N,N,Y,...,L,55.0,1.0,N,N,N,N,0,HEP,2
3,30,M,57,F,N,150,N,Y,Y,N,...,L,187.0,0.0,N,N,N,N,0,HEP,2
4,8,F,83,D,Y,210,Y,Y,Y,C,...,H,189.0,0.0,N,N,N,N,0,HEP,4


In [313]:
# patients 4575
len(obj_1_1)

4575

In [314]:
obj_1_1 = obj_1_1.rename(columns={'RVISINF':"infarct",
                       'RSBP':"bloodpressure" , "OCCODE": "death", "SEX":"sex", "AGE":"age",
                       'STYPE':"strokesubtype", 'TD':"timedeath", "RXHEP":"trt_hep",
                       'RDELAY':"timedelay",'RCONSC':'concious', 'RATRIAL' : 'atrialfibrillation'})


In [315]:
obj_1_1.drop(['RDEF1','RDEF2','RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8', 'DRSISC', 'DRSH', 'DRSUNK','DPE',"trt","RXASP"], inplace=True, axis=1)

In [316]:
obj_1_1.head()

,timedelay,sex,age,concious,infarct,bloodpressure,strokesubtype,RXASP,atrialfibrillation,trt_hep,timedeath,death,recurrentstroke,trt,count_def
0,20,F,79,D,N,175,TACS,N,N,L,29.0,1.0,0,HEP,6
1,24,M,62,F,N,150,TACS,N,N,H,184.0,0.0,0,HEP,3
2,2,F,90,F,N,200,PACS,N,N,L,55.0,1.0,0,HEP,2
3,30,M,57,F,N,150,LACS,N,N,L,187.0,0.0,0,HEP,2
4,8,F,83,D,Y,210,TACS,N,N,H,189.0,0.0,0,HEP,4


In [ ]:
obj_1_1.to_csv('/Users/siboraseranaj/Desktop/project/STA440- Project/HEP_LorH.csv')